Source of data can be obtained [here](https://github.com/CSSEGISandData/COVID-19) - provided by John Hopkins University.

In [1]:
from datetime import date
from datetime import timedelta
from typing import TypeVar
import pandas as pd
pd.options.display.max_rows=5000
DataFrame = TypeVar('pd.core.frame.DataFrame')

In [2]:
def getCoronaVirusDataUSA(iso_date: str, by: str) -> DataFrame:
    """ Obtain counts of confirmed and deaths from SARS-COV-2 / Covid-19 in the U.S. for a single, specific date
    
    Parameter
    ---------
    iso_date : str
        Date in ISO format ('YYYY-MM-DD') for which you want the counts from
    by : str, valid values are 'state' | 'county'
        Indicates that you want counts by state or by county
        
    Returns
    -------
    Pandas dataframe containing confirmed counts by US states
    
    Example Usage
    -------------
    getCoronaVirusDataUSA('2020-03-24', by='state') returns a Pandas dataframe by U.S. state
    getCoroanVirusDataUSA('2020-03-24', by='county') returns a Pandas dataframe by U.S. country
    etCoronaVirusDataUSA('2020-03-24', by='county').filter(like='Ohio', axis='index') filtered to Ohio counties only
    """
    
    data_date: str = date.fromisoformat(iso_date).strftime('%m-%d-%Y')
    data_url: str = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{data_date}.csv"
        
    df: DataFrame = pd.read_csv(data_url)
    df_us: DataFrame = df[df['Country_Region'] == 'US'].query("Confirmed > 0").rename(columns={'Admin2': 'County', 'Combined_Key': 'County_State'})
    
    if 'state' in by:
        confirmed: DataFrame = df_us.groupby('Province_State').agg('sum')['Confirmed'].sort_values(ascending=False).to_frame()
        deaths: DataFrame = df_us.groupby('Province_State').agg('sum')['Deaths'].sort_values(ascending=False).to_frame()
        merged: DataFrame = pd.merge(confirmed, deaths, how='left', left_index=True, right_index=True)
        
        return merged
    elif 'county' in by:
        confirmed: DataFrame = df_us.groupby('County_State').agg('sum')['Confirmed'].sort_values(ascending=False).to_frame()
        deaths: DataFrame = df_us.groupby('County_State').agg('sum')['Deaths'].sort_values(ascending=False).to_frame()
        merged: DataFrame = pd.merge(confirmed, deaths, how='left', left_index=True, right_index=True)
        
        return merged

In [3]:
latest_date = (date.today() + timedelta(days=-1)).strftime("%Y-%m-%d")
latest_date

'2020-03-29'

In [7]:
getCoronaVirusDataUSA(latest_date, by='state')

,Confirmed,Deaths
Province_State,,
New York,59648,806
New Jersey,13386,161
California,5852,123
Michigan,5488,132
Massachusetts,4955,48
Illinois,4596,66
Washington,4465,198
Florida,4246,54
Louisiana,3540,151


In [8]:
getCoronaVirusDataUSA(latest_date, by='county').filter(like='Ohio', axis='index')

,Confirmed,Deaths
County_State,,
"Cuyahoga, Ohio, US",440,3
"Franklin, Ohio, US",247,3
"Summit, Ohio, US",99,5
"Lucas, Ohio, US",94,2
"Hamilton, Ohio, US",85,0
"Mahoning, Ohio, US",84,2
"Lorain, Ohio, US",74,0
"Medina, Ohio, US",45,1
"Miami, Ohio, US",43,5


In [9]:
getCoronaVirusDataUSA(latest_date, by='county').filter(like='New York', axis='index')

,Confirmed,Deaths
County_State,,
"New York City, New York, US",33768,678
"Westchester, New York, US",8519,10
"Nassau, New York, US",6445,39
"Suffolk, New York, US",5023,40
"Rockland, New York, US",2209,8
"Orange, New York, US",1247,5
"Erie, New York, US",414,7
"Dutchess, New York, US",320,2
"Monroe, New York, US",219,5
